# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 4: 

Να βρεθεί το φυλετικό προφίλ των καταγεγραμμένων θυμάτων εγκλημάτων (*Vict Descent*) στο Los Angeles για το έτος 2015, στις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα. 

Να γίνει το ίδιο για τις 3 περιοχές με το χαμηλότερο εισόδημα. 

Να χρησιμοποιήσετε την αντιστοίχιση των κωδικών καταγωγής με την πλήρη περιγραφή από το σύνολο δεδομενων Race and Ethnicity codes. 

Τα αποτελέσματα να τυπωθούν σε δύο ξεχωριστούς πίνακες από το υψηλότερο στο χαμηλότερο αριθμό θυμάτων ανά φυλετικό γκρουπ.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("VictimDescentAnalysis") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1377,application_1732639283265_1339,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ορίζουμε 2 executors, με τα ακόλουθα configurations:
- 1 core  / 2GB memory
- 2 cores / 4GB memory
- 4 cores / 8GB memory

Εκτελούμε κάθε φορά το αντίστοιχο κελί για να επιλέξουμε το κατάλληλο configuration:

In [2]:
conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

#### 1 core / 2GB memory

In [3]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1378,application_1732639283265_1340,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1329,application_1732639283265_1291,pyspark,idle,Link,Link,None,
1330,application_1732639283265_1292,pyspark,idle,Link,Link,None,
1331,application_1732639283265_1293,pyspark,idle,Link,Link,None,
1332,application_1732639283265_1294,pyspark,idle,Link,Link,None,
1333,application_1732639283265_1295,pyspark,idle,Link,Link,None,
1334,application_1732639283265_1296,pyspark,idle,Link,Link,None,
1335,application_1732639283265_1297,pyspark,idle,Link,Link,None,
1337,application_1732639283265_1299,pyspark,idle,Link,Link,None,
1339,application_1732639283265_1301,pyspark,idle,Link,Link,None,
1346,application_1732639283265_1308,pyspark,idle,Link,Link,None,


#### 2 cores / 4GB memory

In [4]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1379,application_1732639283265_1341,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1329,application_1732639283265_1291,pyspark,idle,Link,Link,None,
1330,application_1732639283265_1292,pyspark,idle,Link,Link,None,
1331,application_1732639283265_1293,pyspark,idle,Link,Link,None,
1332,application_1732639283265_1294,pyspark,idle,Link,Link,None,
1333,application_1732639283265_1295,pyspark,idle,Link,Link,None,
1334,application_1732639283265_1296,pyspark,idle,Link,Link,None,
1335,application_1732639283265_1297,pyspark,idle,Link,Link,None,
1337,application_1732639283265_1299,pyspark,busy,Link,Link,None,
1339,application_1732639283265_1301,pyspark,idle,Link,Link,None,
1346,application_1732639283265_1308,pyspark,idle,Link,Link,None,


#### 4 cores / 8GB memory

In [14]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1380,application_1732639283265_1342,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1329,application_1732639283265_1291,pyspark,idle,Link,Link,None,
1330,application_1732639283265_1292,pyspark,idle,Link,Link,None,
1331,application_1732639283265_1293,pyspark,idle,Link,Link,None,
1332,application_1732639283265_1294,pyspark,idle,Link,Link,None,
1333,application_1732639283265_1295,pyspark,idle,Link,Link,None,
1334,application_1732639283265_1296,pyspark,idle,Link,Link,None,
1335,application_1732639283265_1297,pyspark,idle,Link,Link,None,
1337,application_1732639283265_1299,pyspark,idle,Link,Link,None,
1339,application_1732639283265_1301,pyspark,idle,Link,Link,None,
1346,application_1732639283265_1308,pyspark,idle,Link,Link,None,


In [15]:
conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 2
Executor Memory: 8g
Executor Cores: 4

In [16]:
from sedona.spark import *

sedona = SedonaContext.create(spark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Διάβασμα αρχείων εισόδου

In [18]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace, col

income_schema = StructType([
    StructField("Zip Code", StringType()),
    StructField("Community", StringType()),
    StructField("Estimated Median Income", StringType())
])

income_data = spark.read.csv(f"{DATA_BUCKET}/LA_income_2015.csv", header=True, schema=income_schema)

# Μετατροπή του Estimated Median Income σε αριθμητική μορφή
income_data = income_data.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("float")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType
from pyspark.sql.functions import to_timestamp, col, year

# Ορισμός του schema των dataset
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

# Εφόσον μας ενδιαφέρουν μόνο τα δεδομένα για το 2015, διαβάζουμε
# μόνο το 10 dataset (2010-2019)
crime_data = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)

# Κάνουμε parse τα datetime strings του CSV ως timestamps,
# δημιουργούμε νέα στήλη για το έτος στο οποίο συνέβη κάθε έγκλημα
# και μετατρέπουμε τις στήλες LAT, LON σε geometry με το ST_POINT.
# Φιλτράρουμε τα δεδομένα που δεν αφορούν το Null Island (0,0) και
# μόνο τα εγκλήματα του έτους 2015
datetime_format = "MM/dd/yyyy hh:mm:ss a"
crime_data = crime_data \
                .withColumn("geom", ST_Point("LON", "LAT")) \
                .withColumn("Date Rptd", to_timestamp("Date Rptd", datetime_format)) \
                .withColumn("DATE OCC", to_timestamp("DATE OCC", datetime_format)) \
                .withColumn("year", year(col("DATE OCC"))) \
                .filter((col("geom") != ST_Point(0, 0)) & (col("year") == 2015) & col("Vict Descent").isNotNull()) \
                .select("geom", "Vict Descent")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true") \
            .load(f"{DATA_BUCKET}/2010_Census_Blocks.geojson") \
            .selectExpr("explode(features) as features") \
            .select("features.*")

blocks_data = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                    blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type") \
            .filter(col("COMM").isNotNull() & (col("POP_2010") > 0) & (col("CITY") == "Los Angeles")) \
            .select("COMM", "ZCTA10", "POP_2010", "geometry")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
re_codes_schema = StructType([
    StructField("Vict Descent", StringType()),
    StructField("Vict Descent Full", StringType())
])

re_codes = spark.read.csv(f"{DATA_BUCKET}/RE_codes.csv", header=True, schema=re_codes_schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Υλοποίηση με DataFrame

In [24]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών (εύρεση κατά κεφαλήν εισοδήματος)
income_per_person = blocks_data \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        ) \
                        .withColumn("Income per Person", col("Total Income") / col("Population")) \
                        .filter(col("Income per Person").isNotNull()) \
                        .orderBy("Income per Person", ascending=False)

comm_values = [row["COMM"] for row in income_per_person.collect()]
highest_income_areas = comm_values[:3]
lowest_income_areas = comm_values[-3:]

# Σύνδεση Εγκλημάτων / Περιοχών, μόνο για τις 3 περιοχές με το
# υψηλότερο και τις 3 με το χαμηλότερο κατά κεφαλήν εισόδημα
filtered_blocks_data = blocks_data.filter(col("COMM").isin(highest_income_areas+lowest_income_areas))
crimes_per_area = crime_data \
                        .filter(col("Vict Descent").isNotNull()) \
                        .join( \
                              filtered_blocks_data, \
                              ST_Within(crime_data["geom"], filtered_blocks_data["geometry"]), \
                              "inner" \
                        ) \
                        .select("COMM", "Vict Descent")

# Υπολογισμός εγκλημάτων ανά φυλετικό προφίλ,
# για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
# και τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα.
# Δείχνουμε τα αποτελέσματα σε φθίνουσα σειρά αριθμού εγκλημάτων.
crimes_per_ethnicity_highest = crimes_per_area \
                                .filter(col("COMM").isin(highest_income_areas)) \
                                .join(re_codes, on=["Vict Descent"]) \
                                .groupby("Vict Descent Full") \
                                .agg(count("*").alias("#")) \
                                .orderBy("#", ascending=False) \
                                .withColumnRenamed("Vict Descent Full", ("Victim Descent")) \
                                .select("Victim Descent", "#")

crimes_per_ethnicity_lowest = crimes_per_area \
                                .filter(col("COMM").isin(lowest_income_areas)) \
                                .join(re_codes, on=["Vict Descent"]) \
                                .groupby("Vict Descent Full") \
                                .agg(count("*").alias("#")) \
                                .orderBy("#", ascending=False) \
                                .withColumnRenamed("Vict Descent Full", ("Victim Descent")) \
                                .select("Victim Descent", "#")


crimes_per_ethnicity_highest.show(truncate=False)
crimes_per_ethnicity_lowest.show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+---+
|Victim Descent                |#  |
+------------------------------+---+
|White                         |622|
|Other                         |74 |
|Hispanic/Latin/Mexican        |71 |
|Black                         |40 |
|Unknown                       |33 |
|Other Asian                   |18 |
|American Indian/Alaskan Native|1  |
|Chinese                       |1  |
+------------------------------+---+

+------------------------------+----+
|Victim Descent                |#   |
+------------------------------+----+
|Hispanic/Latin/Mexican        |3116|
|Black                         |803 |
|Other                         |769 |
|White                         |740 |
|Korean                        |411 |
|Other Asian                   |397 |
|Unknown                       |383 |
|Filipino                      |11  |
|Chinese                       |8   |
|American Indian/Alaskan Native|5   |
|Pacific Islander              |3   |
|Guamanian             

In [25]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 41.98 seconds

## Αποτελέσματα

| Cores | Memory (GB) | Time (seconds) |
|---|---|---|
| 1 | 2 | 77.20 |
| 2 | 4 | 66.54 |
| 4 | 8 | 41.98 |